In [2]:
from scipy import optimize
import cvxopt
import matplotlib.pyplot as plt
import numpy as np
import sympy
import pandas as pd
sympy.init_printing()
%config InlineBackend.figure_format='svg' #'png2x','svg','jpeg','pdf','png','jpg','retina'

In [26]:
from sklearn import datasets

$$
\text{自定义命令}  
\newcommand{\inner}[2]{{\langle #1,#2\rangle}}
\newcommand{\dlim}{\displaystyle \lim}  
\newcommand{\dint}{\displaystyle \int}   
\newcommand{\ra}{\rangle}  
\newcommand{\la}{\langle}   
\newcommand{\inner}[2]{{\langle #1,#2\rangle}}
\newcommand{\x}{\mathbf{x}}   
\newcommand{\xt}{\mathbf{x}^{\mathsf{T}}}   
\newcommand{\T}{{\mathsf{T}}}
\newcommand{\abf}{\mathbf{a}}   
\newcommand{\abft}{\mathbf{a}^{\mathsf{T}}}   
\newcommand{\R}{\mathbb{R}}   
\newcommand{\C}{\mathbb{C}}   
\newcommand{\E}{\mathrm{e}}   
\newcommand{\F}{\mathbb{F}}   
\newcommand{\X}{\mathbf{X}}   
\newcommand{\Y}{\mathbf{Y}}  
\newcommand{\f}{\mathbf{f}}  
\newcommand{\U}{\mathbf{u}}   
\newcommand{\D}{\mathrm{d}}  
\newcommand{\M}{\mathcal{M}}  
\newcommand{\LL}{\mathcal{L}}  
\newcommand{\nullspace}{\mathrm{null}}   
\newcommand{\range}{\mathrm{range}}   
\newcommand{\Sum}[2]{{\sum_{#1}^{#2}}}   
\newcommand{\Union}[2]{{\bigcup_{#1}^{#2}}}   
\newcommand{\Intersection}[2]{{\bigcap_{#1}^{#2}}}   
\newcommand{\pd}[1]{\frac{\partial}{\partial #1}} \notag
$$  

In [15]:
np.random.seed(1234)
X = np.random.rand(1000)
X.resize((200,5))

# Sample Statistics

Suppose X is the data matrix, 
$$
X = 
\begin{bmatrix}
X_{11} & X_{12} & \cdots & X_{1p} \\ 
X_{21} & X_{22} & \cdots & X_{2p} \\ 
\cdots & \cdots &        & \vdots \\ 
X_{n1} & X_{n2} & \cdots & X_{np} \\  
\end{bmatrix}
$$
then the sample mean and sample covariance matrix of $X$ is 
$$
\bar X = \frac{1}{n} \sum_{a=1}^{n} X_{(a)} = 
\begin{bmatrix}
\bar X_1 \\
\bar X_2 \\
\vdots \\
\bar X_p \\
\end{bmatrix}= \frac{1}{n}X^\prime 1_n
$$

In addition,  considering the deviation matrix
$$
L = 
\begin{bmatrix}
X_{11}-\bar X_1 & X_{12}-\bar X_2 & \cdots & X_{1p}-\bar X_p \\ 
X_{21}-\bar X_1 & X_{22}-\bar X_2 & \cdots & X_{2p}-\bar X_p \\ 
\vdots & \vdots &        & \vdots \\ 
X_{n1}-\bar X_1 & X_{n2}-\bar X_2 & \cdots & X_{np}-\bar X_p \\  
\end{bmatrix}
=(I-\frac{1}{n}1_n 1_n^\prime)X.
$$
and thus
$$
S = \frac{1}{n-1}L^\prime L = \frac{1}{n-1} X^\prime (I-\frac{1}{n}1_n1_n^\prime)X
$$

In [36]:
# diabetes is a dict
iris = datasets.load_iris()

In [37]:
X = iris.data

In [43]:
pd.DataFrame(X)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [77]:
I_n = np.ones(X.shape[0])
I_n.resize((150,1))
E = np.identity(X.shape[0])

In [45]:
np.mean(X,axis=0)

array([5.84333333, 3.05733333, 3.758     , 1.19933333])

In [67]:
(X.T @ I_n)/X.shape[0]

array([[5.84333333],
       [3.05733333],
       [3.758     ],
       [1.19933333]])

In [39]:
np.cov(X,rowvar=False)

array([[ 0.68569351, -0.042434  ,  1.27431544,  0.51627069],
       [-0.042434  ,  0.18997942, -0.32965638, -0.12163937],
       [ 1.27431544, -0.32965638,  3.11627785,  1.2956094 ],
       [ 0.51627069, -0.12163937,  1.2956094 ,  0.58100626]])

In [78]:
(X.T @ (E - (I_n @ I_n.T)/X.shape[0]) @ X)/(X.shape[0]-1)

array([[ 0.68569351, -0.042434  ,  1.27431544,  0.51627069],
       [-0.042434  ,  0.18997942, -0.32965638, -0.12163937],
       [ 1.27431544, -0.32965638,  3.11627785,  1.2956094 ],
       [ 0.51627069, -0.12163937,  1.2956094 ,  0.58100626]])

# Multivariate-Normal 

# Sampling Distribution

## Chisq-distribution and Wishart-distribution

$$
\chi^2(n) \sim Ga(\frac{n}{2},\frac{1}{2})
$$